# Mount the cameras in a convergent configuration:
   - Set up the dual cameras in a configuration where the optical axes converge towards a common point. This convergent setup is essential for capturing images that can be used for phototriangulation.
   - Website: N/A (Instructions only)

# Camera calibration:
   - Perform camera calibration to determine intrinsic camera parameters such as focal length, principal point, and lens distortion coefficients. This step ensures accurate image measurements and helps in subsequent processing steps.
   - OpenCV-Python Tutorials: https://docs.opencv.org/master/dc/dbb/tutorial_py_calibration.html


In [120]:
import numpy as np
import cv2 as cv
import glob

'''
Using camera calibration tutorial avaiable in OpenCV documentation. Link: https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html
'''
def rectify(n):
    # termination criteria
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((8*6,3), np.float32)
    objp[:,:2] = np.mgrid[0:8,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d point in real world space
    imgpoints = [] # 2d points in image plane.

    images = glob.glob(f'C://Users//joao.kreitlon//Documents//IME//2023.1//Foto_digital//Sist_dual//cam_{n}_t//*.png')

    for fname in images:
        img = cv.imread(fname)
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        # Find the chess board corners
        ret, corners = cv.findChessboardCorners(gray, (8,6), None)
        # If found, add object points, image points (after refining them)
        if ret == True:
            objpoints.append(objp)
            corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
            imgpoints.append(corners2)
            # Draw and display the corners
            cv.drawChessboardCorners(img, (8,6), corners2, ret)
            cv.imshow('img', img)
            cv.waitKey(500)
            
    cv.destroyAllWindows()

    #Calibrating the camera:
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

    print(f'\n \n--------------------   CAMERA {n}   --------------------\n')
    print(f'The camera {n} Matrix is the following:\n', mtx)
    print('\nThe distortion coefficients are the following:\n', dist, '\n\n k1,k2,p1,p2,k3, in that order')


    r_list = [list() for _ in range(3)]
    t_list =[list() for _ in range(3)]

    for i in range (len(rvecs)):
        for j in range(len(r_list)):
            r_list[j].append(rvecs[i][j])
            t_list[j].append(tvecs[i][j])

    x0, y0, z0 = t_list
    phi, omega, kappa = r_list


    cont = 0

    for fname in images:
        img = cv.imread(fname)
        h,  w = img.shape[:2]
        newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
        
        # undistort
        dst = cv.undistort(img, mtx, dist, None, newcameramtx)
        # crop the image
        x, y, w, h = roi
        dst = dst[y:y+h, x:x+w]
        name = f'rectif_1//ret_{cont}.png'
        # print(name)
        cv.imwrite(name, dst)
        cont += 1

        rot = [omega, phi, kappa]
        trs = [x0, y0, z0]    

    return mtx, dist, trs, rot, roi

cam_1, cam_2 = rectify(1), rectify(2)


 
--------------------   CAMERA 1   --------------------

The camera 1 Matrix is the following:
 [[1.18713457e+03 0.00000000e+00 2.52493572e+02]
 [0.00000000e+00 1.18908521e+03 2.07691540e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

The distortion coefficients are the following:
 [[-5.74143956e-01  5.35978896e+00  4.35487764e-03 -2.84546899e-03
  -3.66983700e+01]] 

 k1,k2,p1,p2,k3, in that order

 
--------------------   CAMERA 2   --------------------

The camera 2 Matrix is the following:
 [[1.18550643e+03 0.00000000e+00 3.34217203e+02]
 [0.00000000e+00 1.18728572e+03 2.22837815e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

The distortion coefficients are the following:
 [[-4.28091237e-01  1.73926651e-01  3.21650866e-03 -8.07866340e-04
  -1.56390581e+00]] 

 k1,k2,p1,p2,k3, in that order


# External Orientation Parameters

POE: ($\omega$, $\phi$, $\kappa$, $x$, $y$, $z$)

# Rotation Matrix

$$ R_{xyz} = R_x R_y R_z = 
\begin{bmatrix} 1 & 0 & 0 \\ 0 & \cos{\omega} & -\sin{\omega} \\ 1 & \sin{\omega} & \cos{\omega}  \end{bmatrix} 
\begin{bmatrix} \cos{\phi} & 0 & \sin{\phi} \\ 0 & 1 & 0 \\ -\sin{\phi} & 0 & \cos{\phi}  \end{bmatrix} 
\begin{bmatrix} \cos{\kappa} & -\sin{\kappa} & 0 \\ \sin{\kappa} & \cos{\kappa} & 0 \\ 0 & 0 & 1  \end{bmatrix} 

In [89]:
def sub_photo(cam,n,type):

    l = [sub_array[n][0] for sub_array in cam[type]]

    return l


In [90]:
def rot_mtx(o,p,k):
    
    Ro = np.matrix([[1, 0, 0], [0, np.cos(o),-np.sin(o)], [0, np.sin(o),np.cos(o)]])
      
    Rp = np.matrix([[np.cos(p), 0,-np.sin(p)], [0, 1,0], [np.sin(p), 0, np.cos(p)]])
      
    Rk = np.matrix([[np.cos(k),np.sin(k), 0], [-np.sin(k),np.cos(k), 0], [0, 0, 1]])

    M = Rk@Rp@Ro
    return M


# Estimate system mounting parameters:
   Determine the mounting parameters for the dual camera system. These parameters define the geometric relationship between the cameras, such as the distance between them and their relative orientation.
# Annex A

   $$ r_{C_k}^{C_{ref}}(t)= (R^m_{C_{ref}}(t))^{-1} \left( r_{C_k}^{m}(t) - r_{C_{ref}}^m(t) \right) $$

   $$ R^{C_{ref}}_{C_{k}}(t) = R^m_{C_{ref}}(t)^{-1} \cdot R^m_{C_{k}}(t) $$


In [119]:
r_m_Cref = np.vstack([sub_photo(cam_1,3,2)]).T
R_m_Cref = rot_mtx(*sub_photo(cam_1,3,3))

r_m_Ck = np.vstack([sub_photo(cam_2,3,2)]).T
R_m_Ck = rot_mtx(*sub_photo(cam_2,3,3))

# Lever-arm:

r_Cref_Ck = np.linalg.inv(R_m_Cref)@(r_m_Ck-r_m_Cref) 

# Boresight:

R_Cref_Ck = np.linalg.inv(R_m_Cref)@(R_m_Ck) 

print('-'*20, ' BORESIGHT ', '-'*20)
print('\n', R_Cref_Ck ,'\n')

print('-'*20, ' LEVER-ARM ', '-'*20)
print('\n', r_Cref_Ck ,'\n')



--------------------  BORESIGHT  --------------------

 [[ 9.99995150e-01  3.04344646e-03 -6.61104185e-04]
 [-3.09938438e-03  9.93322144e-01 -1.15332183e-01]
 [ 3.05682101e-04  1.15333673e-01  9.93326759e-01]] 

--------------------  LEVER-ARM  --------------------

 [[-2.49285942]
 [-0.28818901]
 [ 0.16816045]] 



In [116]:
R_Cref_Ck

matrix([[ 9.99995150e-01,  3.04344646e-03, -6.61104185e-04],
        [-3.09938438e-03,  9.93322144e-01, -1.15332183e-01],
        [ 3.05682101e-04,  1.15333673e-01,  9.93326759e-01]])